In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\Chcfpp01\Groups\HTS\Code_Repository\Python\Libraries'
Site_Packages = r'c:\users\mr068144\appdata\roaming\python\python36\site-packages'
import sys, os, arcpy, csv, json, math
print(sys.executable)
print(sys.version)
for p in [HSMPY_PATH,Site_Packages]:
    if not p in sys.path:
        sys.path.append(p)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime
from datetime import timedelta
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import zipfile
import shutil
import scipy
from ipywidgets import IntProgress
from time import gmtime, strftime
#from pysal.esda.mapclassify import Natural_Breaks as nb
import matplotlib.patches as patches
print(strftime("%Y-%m-%d %H:%M:%S"))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe
3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
2019-05-16 08:27:15


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'
PythonDir   = Project_DIR + '\\6.Python_Notebook'
DeliverableDir = Project_DIR + '\\8.Deliverable'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

Src_Ped_Loc  = ReceivedDir + '\\Ped_2017_KAB\\crashkabped.csv'
Src_Ped_Unit = ReceivedDir + '\\Ped_2017_KAB\\vehiclekabped.csv'
Src_Ped_Occ  = ReceivedDir + '\\Ped_2017_KAB\\occkabped.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

FOU_2017_105 = ReceivedDir + '\\v105\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'
FOU_2017_10  = ReceivedDir + '\\v10\\jg_request.gdb\\HPMS2017Ownership_Urban_FuncSys'

FHWA_HPMS     = {year:ReceivedDir + '\\FHWA_HPMS\\newmexico{}\\NewMexico{}.shp'.format(year,year) for year in Years}
MS2_HPMS_2017 = {year:ReceivedDir + '\\NM_HPMS_N2017_A{}.csv'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}

StreetNames_SHP = ReceivedDir + "\\streetNames\AltStreetName.shp"

UTM13N   = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

2019-05-16 08:27:15


In [25]:
RSA_DF = pd.DataFrame(columns=['RSA_ID','District','Description','RID','BMP','EMP'])
i = 0
i+=1;RSA_DF.loc[i] = ['RSA_JF',3,'Jefferson','FL-5037-P',0.8,4.1] 
i+=1;RSA_DF.loc[i] = ['RSA_JF',3,'Jefferson','NMX-423-P-5.0-T',0,0.295]
i+=1;RSA_DF.loc[i] = ['RSA_JF',3,'Jefferson','FL-4052-P',2.5,2.7]
i+=1;RSA_DF.loc[i] = ['RSA_R03_Main',3,'D1_US70','US-70-P',148.5,150.1]
i+=1;RSA_DF.loc[i] = ['RSA_R03_Picacho',3,'D1_US70','US-70-P',144,148.07]

Crash_DF.index = Crash_DF.RID_P
RSA_DF.index = RSA_DF.RID
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP)]
    return(df.CID.tolist())
def GetCrashMPs(row):
    df = Crash_DF.loc[[row.RID]]
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP)]
    l = df.MP_P.tolist()
    l.sort()
    return(';'.join(['{:0.2f}'.format(m) for m in l]))
RSA_DF['CIDs'] = RSA_DF.apply(GetCrashCIDs,axis=1)
RSA_DF['Crash_MPs'] = RSA_DF.apply(GetCrashMPs,axis=1)
RSA_DF['NumCrashes'] = RSA_DF.CIDs.apply(len)


RSA_DF

,RSA_ID,District,Description,RID,BMP,EMP,CIDs,Crash_MPs,NumCrashes
RID,,,,,,,,,
FL-5037-P,RSA_JF,3,Jefferson,FL-5037-P,0.8,4.100,"[2013_710113747, 2013_710123506, 2013_71012734...",0.83;0.83;0.83;0.83;1.15;1.19;1.55;1.71;2.20;2...,27
NMX-423-P-5.0-T,RSA_JF,3,Jefferson,NMX-423-P-5.0-T,0.0,0.295,"[2013_30092027, 2013_710095188, 2013_710121134...",0.00;0.00;0.00;0.00;0.00;0.00;0.00;0.00;0.00;0.00,10
FL-4052-P,RSA_JF,3,Jefferson,FL-4052-P,2.5,2.700,"[2013_710124357, 2013_710142290, 2014_71014889...",2.62;2.62;2.62;2.62;2.62;2.62;2.62;2.62;2.62;2...,13
US-70-P,RSA_R03_Main,3,D1_US70,US-70-P,148.5,150.100,"[2013_201304342, 2013_201305359, 2013_20130552...",148.50;148.50;148.58;148.58;148.63;148.70;148....,77
US-70-P,RSA_R03_Picacho,3,D1_US70,US-70-P,144.0,148.070,"[2013_201301741, 2013_201303852, 2013_20130480...",144.03;144.03;144.06;144.50;144.50;144.50;144....,52


In [65]:
path_to_domains = r'\\chcfpp01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\10.Extension\NM_Domains.xlsx'
df = pd.read_excel(path_to_domains)
def TranslateDomains_CSV(DF,DomainCSV,subset=[]):
    df = pd.DataFrame(columns=['field_name'])
    if os.path.basename(DomainCSV).split('.')[-1].lower()=='csv':
        df = pd.read_csv(DomainCSV)
    if os.path.basename(DomainCSV).split('.')[-1].lower() in ['xls','xlsx']:
        df = pd.read_excel(DomainCSV)
    if len(subset)==0:
        subset = DF.columns
    else:
        subset = [c for c in subset if c in DF.columns]
    for c in subset:
        if c in df.field_name.unique():
            idf = df[df.field_name=='Class'][['code','description']]
            idf.index = idf.code
            d = idf.description.to_dict()
            if len(d)>0:
                DF[c] = DF[c].apply(lambda x:'{}. {}'.format(x,d[x]) if x in d.keys() else x)
    return(DF)
Crash_DF['Class_T'] = TranslateDomains_CSV(Crash_DF[['Class']].fillna(99).astype(int),path_to_domains)
Crash_DF['Weather_T'] = TranslateDomains_CSV(Crash_DF[['Weather']].fillna(99).astype(int),path_to_domains)
Crash_DF.index= Crash_DF.CID
RSA_DF.index = RSA_DF.RSA_ID
Crash_DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in Crash_DF.Date]
Crash_DF['DayName'] = [d.weekday_name for d in Crash_DF.Date]
Crash_DF['Month']  = [d.strftime('%b') for d in Crash_DF.Date]
Crash_DF['KAB'] = 1
TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
F = ['PEDinv','PECinv','Druginv','Alcinv','MCinv','TRKinv','HZinv','OnCurve','TribalJurisdiction','Int_Related']
Des =  ['Pedestrian Involved', 'Pedalcyclist Involved','Drug Involved','Alcohol Involved','Motorcylce Involved','Truck Involved',
'Hazardous Material Involved','On the Curve','Tribal Jurisdiction','Intersection Related']

for i in set(RSA_DF.index.tolist()):
    print(i)
    try:
        L = RSA_DF.loc[i]['CIDs'].tolist()
        L = [i for l in L for i in l]
    except:
        L = RSA_DF.loc[i]['CIDs']
    print(L)
    r = ''
    SHSP_EABarStateLine(Crash_DF,L,'Time','Site_{}_Time.png'.format(i),Order=TimeOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,L,'DayName','Site_{}_Day.png'.format(i),Order=DayOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,L,'Month','Site_{}_Month.png'.format(i),Order=MonthOrder,Length=0,r=r)
    #SHSP_EABarStateLine(Crash_DF,L,'DayNight','Site_{}_DayNight.png'.format(i),Order=['Dawn','Day','Dusk','Night'],Length=0,r=r)
    #SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Light','Light.png',Order=[],Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,L,'Weather_T','Site_{}_Weather.png'.format(i),Order=[],Length=0,r=r)
    CrashTypeSeverity(Crash_DF,L,'Class_T','KABCO',['K','A','B'], PNGOut='Site_{}_TypeSeverity.png'.format(i),
                  title='{}'.format(i))
    EASeverity(Crash_DF,L,F,Des,SeverityField='KABCO',Order = ['K','A','B'], PNGOut='Site_{}_EASeverity.png'.format(i),
               title='{}'.format(i))
    Crash_DF.loc[L].to_csv('{}.csv'.format(i))

RSA_JF
['2013_710113747', '2013_710123506', '2013_710127347', '2013_710141876', '2013_710147765', '2014_710150400', '2014_710150915', '2014_710152239', '2014_710183026', '2014_710186328', '2014_710206381', '2015_710214866', '2015_710216239', '2015_710236234', '2015_710249222', '2015_710252894', '2015_710255299', '2015_710256222', '2015_710266474', '2015_9999925', '2016_710211691', '2016_710274248', '2016_710274257', '2012_710054699', '2012_710113448', '2012_710117159', '2012_710119926', '2013_30092027', '2013_710095188', '2013_710121134', '2013_710129330', '2013_710139231', '2012_710054187', '2012_710085363', '2012_710113190', '2012_710115116', '2012_710116067', '2013_710124357', '2013_710142290', '2014_710148899', '2014_710150900', '2014_710191236', '2014_710202061', '2015_710257116', '2016_710365389', '2016_710365400', '2012_710056664', '2012_710058818', '2012_710118709', '2012_710124520']


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RSA_R03_Main
['2013_201304342', '2013_201305359', '2013_201305520', '2013_201306868', '2013_201307472', '2013_201309840', '2013_201310954', '2013_201311402', '2013_201311683', '2013_201312228', '2013_201313087', '2013_201319959', '2013_201325873', '2013_201326103', '2013_201328358', '2014_201400484', '2014_201401389', '2014_201405708', '2014_201405899', '2014_201409035', '2014_201414099', '2014_201414250', '2014_201417946', '2014_801421298', '2014_801421339', '2014_801424150', '2014_801424311', '2014_801424910', '2014_801425815', '2014_801426914', '2014_801427914', '2015_801501649', '2015_801503100', '2015_801503738', '2015_801503878', '2015_801512538', '2015_801512950', '2015_801516743', '2015_801518786', '2015_801520188', '2015_801520194', '2015_801520449', '2015_801520792', '2015_801524165', '2015_801524896', '2015_801526438', '2015_801527121', '2015_801527526', '2015_801529031', '2016_801603563', '2016_801603879', '2016_801605263', '2016_801608096', '2016_801610078', '2016_80161561

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


RSA_R03_Picacho
['2013_201301741', '2013_201303852', '2013_201304803', '2013_201309759', '2013_201312440', '2013_201315070', '2013_201316278', '2013_201316776', '2013_201317248', '2013_201323391', '2013_201327962', '2013_30064930', '2013_30094405', '2014_201409284', '2014_201409917', '2014_201410489', '2014_201411806', '2014_201415097', '2014_201416451', '2014_201417600', '2014_30103320', '2014_30107103', '2014_710105690', '2014_710106315', '2014_801427203', '2014_801427796', '2015_30140837', '2015_801502850', '2015_801514636', '2015_801518054', '2015_801519455', '2015_801519790', '2015_801522355', '2015_801523972', '2016_710159213', '2016_710290327', '2016_801601535', '2016_801607914', '2016_801611083', '2016_801612011', '2016_801614166', '2016_801615371', '2016_801617232', '2016_801628024', '2012_201204593', '2012_201215363', '2012_201219355', '2012_201220910', '2012_201221516', '2012_30043514', '2012_30043704', '2012_710069601']


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [64]:
Crash_DF.loc[L].to_csv('temp.csv')

In [59]:
TranslateDomains_CSV(Crash_DF[['Weather']].fillna(99).astype(int),path_to_domains)


NM_Crash_KAB_12_16,Weather
CID,
2013_10045602,1
2013_10391228,1
2013_10394854,1
2013_10395867,1
2013_10398143,1
2013_10398193,1
2013_10398199,1
2013_10398212,1
2013_10398220,1


In [75]:
#CID_DF = hsmpy3.common.FCtoDF(ResultsGDB+'\\R_Agg_Crash')
S = CID_DF.groupby('Site_ID')['CID'].aggregate(list)
for i,s in S.iteritems():
    print(i)
    Crash_DF.loc[s].to_csv('Site_{}_RawCrash.csv'.format(i))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
Crash_DF['YEAR'] = Crash_DF.Date.apply(lambda d:d.year)
Crash_DF.YEAR.value_counts()

2016    5048
2015    4903
2014    4398
2012    4143
2013    4069
2017     342
Name: YEAR, dtype: int64

In [53]:
# Reading Source Crashes
print(strftime("%Y-%m-%d %H:%M:%S"))

Crash_DF = hsmpy3.common.FCtoDF(Project_GDB + '\\NM_Crash_KAB_12_16',readGeometry=True)
Crash_DF.index = Crash_DF.CID

Loc_DF = pd.read_csv(Src_Loc,low_memory=False)
Loc_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Loc_DF.year,Loc_DF.UCRnumber)]
Loc_DF.index = Loc_DF.CID
Loc_DF = Loc_DF.sort_index()

F = ['CrashDirection',
 'Astreet',
 'Bstreet',
 'Landmark',
 'DirectionFromLandmark',
 'Measurement',
 'MeasurementUnit',
 'Class',
 'Analysis',
 'CrashOccurrence',
 'HitRun',
 'Light',
 'Weather',
 'RoadCharacter',
 'RoadGrade',
 'nVeh',
 'TopCFacc',
 'Alcinv',
 'Druginv',
 'MCinv',
 'PECinv',
 'PEDinv',    
 'TRKinv',
 'HZinv',
 'TribalJurisdiction']
Loc_DF.index = Loc_DF.CID
for c in F:
    Crash_DF[c] = Loc_DF[c].loc[Crash_DF.index]
Crash_DF['OnCurve'] = Crash_DF.RoadCharacter.apply(lambda x:1 if x==2 else 0)
print(Crash_DF.shape)

Unit_DF = pd.read_csv(Src_Unit,low_memory=False)
Unit_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)]
Unit_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Unit_DF.year, Unit_DF.UCRnumber)]
#Unit_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v) for y,u,v in zip(Unit_DF.year, Unit_DF.UCRnumber,Unit_DF.VehNo)],names=['CID','Unit'])
Unit_DF.index = Unit_DF.CID
Unit_DF = Unit_DF.sort_index()
Unit_DF = Unit_DF[Unit_DF.CID.isin(Crash_DF.CID)]
Unit_DF['Shape'] = Crash_DF.Shape.loc[Unit_DF.index]
Unit_DF['Date'] = Crash_DF.Date.loc[Unit_DF.index]
F = ['VehNo','Date', 'Passengers', 'DALC', 'DRUG', 'DRESID', 'TopCFcar', 'TypeV', 'Belt', 'Helmet', 'VehDirection', 'StreetOn', 'PostedSpeed', 'SafeSpeed', 'LeftScene', 'DrSeatPos', 'DrAge', 'DrSex', 'DrInjuryCode', 'DrOPCode', 'DrOPProperlyUsed', 'DrAirbagDeployed', 'DrEjected', 'DrEMSNum', 'DrMedTrans', 'DLState', 'DLType', 'DLRestrictions', 'DLExpires', 'DLEndorsements', 'DLStatus', 'VeYear', 'VeMake', 'VeColor', 'VeBodystyle', 'VeCargoBody', 'VeUse1', 'VeUse2', 'VeLicPlateRegYr', 'VeLicPlateState', 'VeTowed', 'VeTowedDisabled', 'VeDamageSeverity', 'VeDamageExtent', 'VeDamageAll', 'HazmatName', 'HazmatPlacard', 'HazmatReleased', 'RoadConditionsVe', 'RoadSurfaceVe', 'TrafficControlDevice', 'RoadDesignLanes', 'RoadDesignDivider', 'RoadDesign', 'SequenceEvent1', 'SequenceEvent2', 'SequenceEvent3', 'SequenceEvent4', 'vVehNo', 'vViolation', 'vAction', 'TraCS', 'CUID', 'CID', 'Shape']
Unit_DF = Unit_DF[F]
print(Unit_DF.shape)


OL = ['CUID','PPLNo', 'OccNo', 'SeatPos', 'Age', 'Sex', 'Injury', 'OPCode', 'OPProperlyUsed', 'AirbagDeployed', 'Ejected', 'MedTrans', 'DAparked', 'Belt', 'Helmet']
Occ_DF = pd.read_csv(Src_Occ,low_memory=False)
Occ_DF['CID'] = ['{}_{}'.format(y,u) for y,u in zip(Occ_DF.year, Occ_DF.UCRnumber)]
Occ_DF['CUID'] = ['{}_{}_{}'.format(y,u,v) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)]
#Occ_DF.index = pd.MultiIndex.from_tuples([('{}_{}'.format(y,u),v,p) for y,u,v,p in zip(Occ_DF.year, Occ_DF.UCRnumber,Occ_DF.VehNo,Occ_DF.PPLNo)],names=['CID','Unit','Person'])
Occ_DF.index = Occ_DF.CID
Occ_DF = Occ_DF[Occ_DF.CID.isin(Crash_DF.CID)]
Occ_DF = Occ_DF.sort_index()
Occ_DF['Shape'] = Crash_DF.Shape.loc[Occ_DF.index]
Occ_DF['Date'] = Crash_DF.Date.loc[Occ_DF.index]
F = ['VehNo','Date', 'PPLNo', 'OccNo', 'SeatPos', 'Age', 'Sex', 'Injury', 'OPCode', 'OPProperlyUsed', 'AirbagDeployed', 'Ejected', 'MedTrans', 'DAparked', 'Belt', 'Helmet', 'CID', 'CUID', 'Shape']
Occ_DF = Occ_DF[F]
print(Occ_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-05-16 10:26:23
(22903, 52)
(40141, 65)
(59513, 19)
2019-05-16 10:26:53


In [5]:
# Reading the Final Network Screening Results
print(strftime("%Y-%m-%d %H:%M:%S"))
SHP_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\8.Deliverable\Shapefiles'
Ped_FC = SHP_Dir + '\\State_Pedestrian.shp' 
Seg_FC = SHP_Dir + '\\State_Segment.shp' 
Int_FC = SHP_Dir + '\\State_Int.shp'
IntD_DF = hsmpy3.common.FCtoDF(Int_FC,readGeometry=True)
SegD_DF = hsmpy3.common.FCtoDF(Seg_FC,readGeometry=True)
PedD_DF = hsmpy3.common.FCtoDF(Ped_FC,readGeometry=True)
SegD_DF['Mileage'] = SegD_DF.Shape.apply(lambda x:x.length/5280.0)
def TranslateUrban(x):
    codes = {0:'Rural',
            1171:'Albuquerque',
            27253:'El Paso (Partial)',
            29278:'Farmington',
            47935:'Las Cruces',
            51499:'Los Lunes',
            79363:'Santa Fe',
            99998:'Small Urban',
            99999:'Rural'}
    return(codes[x])
SegD_DF['UrbanRural'] = SegD_DF.URBAN_CODE.apply(TranslateUrban)
PedD_DF['UrbanRural'] = PedD_DF.URBAN_CODE.apply(TranslateUrban)
IntD_DF['UrbanRural'] = IntD_DF.I_URBANCOD.apply(TranslateUrban)

SegD_DF['StateLocal'] = SegD_DF.OWNERSHIP.apply(lambda x:'State' if x==1 else 'Other')
PedD_DF['StateLocal'] = PedD_DF.OWNERSHIP.apply(lambda x:'State' if x==1 else 'Other')
IntD_DF['StateLocal'] = IntD_DF.apply(lambda row:'State' if (row.OWNERSHIP1==1 or row.OWNERSHIP_==1) else 'Other',axis=1)


SegD_DF['Dis_Label'] = SegD_DF.apply(lambda row:'{} ({})'.format(row.RID.replace('-',''),row.RoadLabel),axis=1)
PedD_DF['Dis_Label'] = PedD_DF.apply(lambda row:'{} ({})'.format(row.RID.replace('-',''),row.RoadLabel),axis=1)

PedD_DF['Mileage'] = PedD_DF.Shape.apply(lambda x:x.length/5280.0)
IntD_DF.index = IntD_DF.Int_ID
IntD_DF = IntD_DF.sort_values('CF_Obs',ascending=False)
SegD_DF = SegD_DF.reset_index()
idx = pd.IntervalIndex.from_tuples([(bmp,emp) for bmp,emp in zip(SegD_DF.BMP,SegD_DF.EMP)],'left')
SegD_DF.index = pd.MultiIndex.from_arrays([SegD_DF.RID.apply(lambda x:x.replace('-','')),idx],names = ['Route ID','Milepost'])
SegD_DF = SegD_DF.sort_index()
IntD_DF = IntD_DF[IntD_DF.District.apply(len)>1]
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-26 09:54:57
2019-04-26 09:55:41


In [3]:
# Create Results GDB
ResultsGDB = DeliverableDir + '\\GDB3.gdb'
hsmpy3.common.ListFCinGDBorMDB(ResultsGDB)
Crash_DF = hsmpy3.common.FCtoDF(ResultsGDB+'\\NM_Crash_KAB_12_16')
print(Crash_DF.shape)

(22903, 63)


In [13]:
HTML(Crash_DF.iloc[[123]].T.sort_index().to_html())

OBJECTID,124
NM_Crash_KAB_12_16,
Alcinv,0
Analysis,402
Astreet,DENNIS CHAVEZ BLVD SW
Bstreet,CONDERSHIRE RD SW
CID,2013_10400335
Class,4
ClassA,0
ClassB,1
Count,1


In [7]:
# Create Crash DataFrames
Crash_DF.index = Crash_DF.RID_P
SegD_DF.index = SegD_DF.SiteID
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP) & (df.Int_Related==0)]
    return(df.CID.tolist())
SegD_DF['CIDs'] = SegD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.SiteID,cid) for i,r in SegD_DF.iterrows() for cid in r.CIDs],columns=['SiteID','CID'])
df.index = df.SiteID
SegD_Crash_DF = df.sort_index()

PedD_DF.index = PedD_DF.SiteID
Crash_DF['PedInv'] = Crash_DF.apply(lambda row:sum([row.PedK,row.PedA,row.PedB])>0,axis=1)
def GetCrashCIDs(row):
    df = Crash_DF.loc[[row.RID]]
    #display(df)
    df = df[(df.MP_P>=row.BMP) & (df.MP_P<=row.EMP) & (df.PedInv)]
    return(df.CID.tolist())
PedD_DF['CIDs'] = PedD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.SiteID,cid) for i,r in PedD_DF.iterrows() for cid in r.CIDs],columns=['SiteID','CID'])
df.index = df.SiteID
PedD_Crash_DF = df.sort_index()

IntD_DF.index = IntD_DF.Int_ID
def GetCrashCIDs(row):
    df = Crash_DF[(Crash_DF.Int_ID==row.Int_ID)]
    return(df.CID.tolist())
IntD_DF['CIDs'] = IntD_DF.apply(GetCrashCIDs,axis=1)
df = pd.DataFrame([(r.Int_ID,cid) for i,r in IntD_DF.iterrows() for cid in r.CIDs],columns=['Int_ID','CID'])
df.index = df.Int_ID
IntD_Crash_DF = df.sort_index()
Crash_DF.index = Crash_DF.CID

In [8]:
# Read Segments - used for Int App Shapes
print('[{}] read segments'.format(strftime("%Y-%m-%d %H:%M:%S")))
Seg_2016 = hsmpy3.common.FCtoDF(Segs[2016])
Seg_2016.index = Seg_2016.ROUTE_ID.apply(lambda x:x.replace('-',''))
print('[{}] - {}'.format(strftime("%Y-%m-%d %H:%M:%S"),Seg_2016.shape))

[2019-04-26 09:56:09] read segments
[2019-04-26 10:02:55] - (224751, 17)


In [9]:
# Create Int App Data Frame
def extractLabel(x):
    try:
        M = x.split(';')
        N = []
        for s in M:
            L = s.split('@')
            a = L[1].find('(')+1
            b = L[1][a:-1]
            c = float(L[1][:a-1])
            a = L[0]
            N.append(['{} ({})'.format(a,b),c,a])
        return(N)
    except:
        return([])
        print(x)
def QuerySeg1(row):
    df = Seg_2016.loc[row.RID]
    df = df[(df.BEG_POINT<=row.MP) & (df.END_POINT>=row.MP)]
    try:
        return(list(df.OWNERSHIP)[0])
    except:
        pass
def QuerySeg2(row):
    df = Seg_2016.loc[row.RID]
    df = df[(df.BEG_POINT<=row.MP) & (df.END_POINT>=row.MP)]
    try:
        return(list(df.URBAN_CODE)[0])
    except:
        pass

def TranslateUrban(x):
    codes = {0:'Rural',
            1171:'Albuquerque',
            27253:'El Paso (Partial)',
            29278:'Farmington',
            47935:'Las Cruces',
            51499:'Los Lunes',
            79363:'Santa Fe',
            99998:'Small Urban',
            99999:'Rural'}
    return(codes[x])

print('[{}] Create Intersection Approach Dataframe'.format(strftime("%Y-%m-%d %H:%M:%S")))
df = IntD_DF
sdf = pd.DataFrame(df.ApprLabels.apply(extractLabel))
sdf['RIDList'] = IntD_DF.RIDList.apply(lambda x:';'.join(list(set([i for i in x.split(';') if i[-1]=='P'])))).loc[sdf.index]
sdf = sdf.reset_index()
cdf = pd.DataFrame(index = pd.Index([b[0] for i in sdf.ApprLabels for b in i],name='Label'),
                   data={'Int_ID':[r.Int_ID for i,r in sdf.iterrows() for x in r.ApprLabels],
                         'RID'   :[b[2] for i in sdf.ApprLabels for b in i],
                         'MP'    :[b[1] for i in sdf.ApprLabels for b in i],
                        })

cdf['StateLocal'] = cdf.apply(QuerySeg1,axis=1).fillna(0).astype(int).apply(lambda x:'State' if x==1 else 'Other')
cdf['UrbanRural'] = IntD_DF.loc[cdf.Int_ID.tolist(),'UrbanRural'].tolist()
cdf['District'] = IntD_DF.loc[cdf.Int_ID.tolist(),'District'].tolist()
cdf['Shape'] = IntD_DF.loc[cdf.Int_ID.tolist(),'Shape'].tolist()
cdf.index.name = 'Dis_Label'
cdf['CIDs'] = IntD_DF.CIDs.loc[cdf.Int_ID.tolist()].tolist()
cdf['SafetyTier'] = IntD_DF.SafetyTier.loc[cdf.Int_ID.tolist()].tolist()
def Fun1(row):
    rid = row.RID
    int_id = row.Int_ID
    ridL = sdf.loc[int_id,'RIDList'].split(';')
    #return(ridL)
    for r in ridL:
        if r.replace('-','')==rid:
            return(r)
sdf.index = sdf.Int_ID
cdf['ROUTE_ID'] = cdf.apply(Fun1,axis=1)
cdf = cdf.reset_index()
IntApp_DF = cdf

df = IntApp_DF[['ROUTE_ID','MP','Int_ID']]
df['BMP'] = df.MP-250.0/5280.0
df['EMP'] = df.MP+250.0/5280.0
df['BMP'] = df.BMP.apply(lambda x:0 if x<0 else x)
df = df.rename(columns={'RID':'ROUTE_ID','BMP':"BEG_POINT",'EMP':'END_POINT'})
hsmpy3.common.ConvertPandasDFtoTable(DF=df,Table=Project_GDB+'\\Int_App_Table')
hsmpy3.network.CreateRouteEventLayer(AttTable=Project_GDB+'\\Int_App_Table',
                                     Sites_Routes=Routes[2016],
                                     BMP='BEG_POINT',EMP='END_POINT',RouteID='ROUTE_ID',
                                     Fields=['Int_ID'],Output=Project_GDB+'\\Int_App_Event')
hsmpy3.common.PrintSummary(Project_GDB+'\\Int_App_Event')
df = hsmpy3.common.FCtoDF(Project_GDB+'\\Int_App_Event',readGeometry=True)
s = df.groupby('Int_ID').Shape.aggregate(lambda L:arcpy.Polyline(arcpy.Array([l[0] for l in L])))
IntApp_DF.index = IntApp_DF.Int_ID
IntApp_DF['Shape'] = s.loc[IntApp_DF.index]
print('[{}] - {}'.format(strftime("%Y-%m-%d %H:%M:%S"),IntApp_DF.shape))

[2019-04-26 10:02:55] Create Intersection Approach Dataframe
[2019-04-26 10:04:02] create table


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

[2019-04-26 10:04:02] deleting current table: 11178
[2019-04-26 10:07:57] adding fields
[2019-04-26 10:10:55] inserting rows
[2019-04-26 10:11:21] done! 11178
Type: Polyline
Columns: 8 x Rows: 10458
['BEG_POINT', 'END_POINT', 'Int_ID', 'LOC_ERROR', 'OBJECTID', 'ROUTE_ID', 'Shape', 'Shape_Length']
[2019-04-26 10:18:23] - (11178, 11)


In [113]:
Res_V3 = pd.read_excel(Project_DIR+'\\10.Extension\\Aggregated_Sites_V3.xlsx',index_col=[0,1,2,3])
Res_DF['Selection'] = Res_V3['Selection For Further Review'].loc[Res_DF.index]


In [114]:
# Add aggregated results to GDB
df = Res_DF[['Site_ID','Shape','Selection','PSI_Mileage','Ped_Mileage','Num_Int','Total_Mileage'] + ['{}{}'.format(t,s) for t in ['Total_','Seg','Int','Ped'] for s in ['K','A','B']]]
df.Ped_Mileage = df.Ped_Mileage.fillna(0).round(2)
for c in ['{}{}'.format(t,s) for t in ['Seg','Int','Ped'] for s in ['K','A','B']]:
    df[c] = df[c].fillna(0).astype(int)
df = df.reset_index()
hsmpy3.common.ConvertPandasDFtoTable(df,ResultsGDB+'\\Agg_Locations',shape='Shape',spatialReference=hsmpy3.nm.NAD83_NM)

R_Agg_Crash = ResultsGDB + '\\R_Agg_Crash'
arcpy.Delete_management(R_Agg_Crash)
arcpy.management.CreateRelationshipClass(
    origin_table           = ResultsGDB + '\\Agg_Locations',
    destination_table      = ResultsGDB + '\\NM_Crash_KAB_12_16', 
    out_relationship_class = R_Agg_Crash, 
    relationship_type      = "SIMPLE", 
    forward_label          = "Crash", 
    backward_label         = "Agg_Locations", 
    message_direction      = "NONE", 
    cardinality            = "MANY_TO_MANY", 
    attributed             = "NONE", 
    origin_primary_key     = "Site_ID", 
    origin_foreign_key     = "Site_ID", 
    destination_primary_key= "CID", 
    destination_foreign_key= 'CID'
)

df = pd.DataFrame([(r.Site_ID,cid) for i,r in Res_DF.iterrows() for cid in r.All_CIDs],columns=['Site_ID','CID'])
df.index = df.Site_ID
Agg_Crash_DF = df
uc = arcpy.InsertCursor(R_Agg_Crash)
for i,r in Agg_Crash_DF.iterrows():
    row = uc.newRow()
    row.setValue('Site_ID',r.Site_ID)
    row.setValue('CID',r.CID)
    uc.insertRow(row)
del uc
print('[{}] Agg - Crash done!'.format(strftime("%Y-%m-%d %H:%M:%S")))

[2019-04-17 12:18:30] create table
[2019-04-17 12:18:30] deleting current table: 447
[2019-04-17 12:18:38] spatial reference passed: NAD_1983_2011_StatePlane_New_Mexico_Central_FIPS_3002_Ft_US
[2019-04-17 12:18:38] geometry detected: POLYLINE
[2019-04-17 12:18:41] adding fields
[2019-04-17 12:19:53] inserting rows
[2019-04-17 12:19:55] done! 447
[2019-04-17 12:20:08] Agg - Crash done!


In [11]:
# Create Figures for each Aggreagted Site
print(strftime("%Y-%m-%d %H:%M:%S"))
def SHSP_EABarStateLine(Crash_DF,Selection,Field,png_out,Order=[],Length=0,r=''):
    df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
    df.KAB = df.KAB/float(sum(df.KAB))
    df.columns = [Field]
    if len(Order) == 0:
        df = df.sort_index(ascending=True)
    else:
        df = df.loc[Order]
    if 1<2:
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        eadf = pd.DataFrame(Crash_DF.loc[Selection].groupby([Field])['KAB'].aggregate(sum))
        eadf = eadf.fillna(0)
        eadf.KAB = eadf.KAB/float(sum(eadf.KAB))
        eadf.columns = [Field]
        if Length>0:
            eadf = eadf.sort_values(Field,ascending=False)
            eadf = eadf.iloc[range(min(Length,len(eadf)))]
            df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
            df.KAB = df.KAB/float(sum(df.KAB))
            df.columns = [Field]
            if len(Order) == 0:
                df = df.sort_index(ascending=True)
            else:
                df = df.loc[Order]
            df  = df.loc[list(eadf.index)]
        else:
            eadf = eadf.loc[df.index]
        eadf = eadf.fillna(0)
        V = [v1-v2 for v1,v2 in zip(eadf[Field],df[Field])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(eadf)),eadf[Field],align='center',color=my_cmap(my_norm(V)))
        plt.xticks(range(len(df)),df.index,rotation=90)
        p2, = plt.plot(range(len(df)),df[Field],'-o',color='green')
        plt.xlabel(Field)
        #plt.xticks(rotation=90)
        plt.title('{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
        plt.grid()
        plt.gca().set_yticklabels(['{:,.2%}'.format(x) for x in plt.gca().get_yticks()])
        plt.legend([p2,p1],['Statewide Crashes ({})'.format(Crash_DF.shape[0]),'Site-Specific Crashes ({})'.format(len(Selection))],loc=2,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.tight_layout()
        plt.savefig(png_out,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close()
def SHSP_TimeTrend_Contour(DF,PNGName,Title):
    import warnings
    warnings.filterwarnings('ignore')
    import matplotlib.pyplot as plt
    DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in DF.Date]
    TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
    TimeOrder.reverse()
    DF['DayName'] = [d.weekday_name for d in DF.Date]
    DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    DF['Month']  = [d.strftime('%b') for d in DF.Date]
    MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    if 1<2:
        df0 = pd.DataFrame(DF.groupby(['Month','DayName','Time'])['KAB'].aggregate(sum))
        df0 = df0.unstack('Time').T
        df0.index = df0.index.droplevel(0)
        arr = []
        for m in MonthOrder:
            for d in DayOrder:
                arr.append((m,d))
        cs = pd.MultiIndex.from_tuples(arr)
        df = pd.DataFrame(columns=cs,index=TimeOrder)
        for i1 in df.index:
            for j1 in df.columns:
                try:
                    df.set_value(i1,j1,int(df0.loc[i1,j1]))
                except:
                    pass
        df = df.fillna(0)
        df = df.loc[TimeOrder]
        L = []
        for l in list(df.values):
            L.extend(l)
        v_max = max(L)
        v_min = min(L)
        df = df/v_max
        v_max = 1
        v_min = 0
    
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        ax1 = plt.subplot(211)
    
        df1 = df[MonthOrder[0:6]]
        #pl1 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl1 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        xposition = [7*(i1)-0.5 for i1 in range(1,6)]
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[1][0:2] + '  ' + f[0],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.gca().xaxis.tick_top()
        plt.yticks(range(1,len(df1.index)),list(df1.index)[1:],rotation=0,fontsize=6)
        
        plt.grid()
        df1 = df[MonthOrder[6:13]]
        ax2 = plt.subplot(212)
        #pl2 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl2 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[0] + '  ' + f[1][0:2],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.yticks(range(len(df1.index)),df1.index,rotation=0,fontsize=6)
        plt.grid()
        #plt.gcf().subplots_adjust(right=0.8)
        #cbar_ax = plt.gcf().add_axes([0.85, 0.15, 0.05, 0.7])
        #plt.colorbar(pl1, cax=cbar_ax)
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.suptitle(Title,y=0.99)
        #plt.tight_layout()
        plt.savefig(PNGName,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close('all')
        del ax1
        del ax2
        del pl1
        del pl2

f = IntProgress(min=0,max=Res_DF.shape[0],value=0)
display(f)
for i,r in Res_DF.iterrows():
    SHSP_TimeTrend_Contour(Crash_DF.loc[r.All_CIDs],'Site_{}_Contour.png'.format(r.Site_ID),'{} - {} - {} - {}\n({} KAB Crashes)'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label,Crash_DF.loc[r.All_CIDs].shape[0]))
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Time','Site_{}_Time.png'.format(r.Site_ID),Order=TimeOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'DayName','Site_{}_Day.png'.format(r.Site_ID),Order=DayOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Month','Site_{}_Month.png'.format(r.Site_ID),Order=MonthOrder,Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'DayNight','Site_{}_DayNight.png'.format(r.Site_ID),Order=['Dawn','Day','Dusk','Night'],Length=0,r=r)
    #SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Light','Light.png',Order=[],Length=0,r=r)
    SHSP_EABarStateLine(Crash_DF,r.All_CIDs,'Weather','Site_{}_Weather.png'.format(r.Site_ID),Order=[],Length=0,r=r)
    f.value += 1
print(strftime("%Y-%m-%d %H:%M:%S"))

    CrashTypeSeverity(Crash_DF,Res_DF.All_CIDs.loc[i],'Class_T','KABCO',['K','A','B'], PNGOut='Site_{}_TypeSeverity.png'.format(r.Site_ID),
                  title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
    EASeverity(Crash_DF,Res_DF.All_CIDs.loc[i],F,Des,SeverityField='KABCO',Order = ['K','A','B'], PNGOut='Site_{}_EASeverity.png'.format(r.Site_ID),
               title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))


2019-04-26 10:18:39


IntProgress(value=0, max=447)

MemoryError: 

MemoryError: 

<Figure size 5200x2800 with 2 Axes>

In [45]:
# Create Figures for each Aggreagted Site
print(strftime("%Y-%m-%d %H:%M:%S"))
def SHSP_EABarStateLine(Crash_DF,Selection,Field,png_out,Order=[],Length=0,r=''):
    df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
    df.KAB = df.KAB/float(sum(df.KAB))
    df.columns = [Field]
    if len(Order) == 0:
        df = df.sort_index(ascending=True)
    else:
        df = df.loc[Order]
    if 1<2:
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        eadf = pd.DataFrame(Crash_DF.loc[Selection].groupby([Field])['KAB'].aggregate(sum))
        eadf = eadf.fillna(0)
        eadf.KAB = eadf.KAB/float(sum(eadf.KAB))
        eadf.columns = [Field]
        if Length>0:
            eadf = eadf.sort_values(Field,ascending=False)
            eadf = eadf.iloc[range(min(Length,len(eadf)))]
            df = pd.DataFrame(Crash_DF.groupby([Field])['KAB'].aggregate(sum))
            df.KAB = df.KAB/float(sum(df.KAB))
            df.columns = [Field]
            if len(Order) == 0:
                df = df.sort_index(ascending=True)
            else:
                df = df.loc[Order]
            df  = df.loc[list(eadf.index)]
        else:
            eadf = eadf.loc[df.index]
        eadf = eadf.fillna(0)
        V = [v1-v2 for v1,v2 in zip(eadf[Field],df[Field])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(eadf)),eadf[Field],align='center',color=my_cmap(my_norm(V)))
        plt.xticks(range(len(df)),df.index,rotation=90)
        p2, = plt.plot(range(len(df)),df[Field],'-o',color='green')
        plt.xlabel(Field)
        #plt.xticks(rotation=90)
        plt.title(r)
        plt.grid()
        plt.gca().set_yticklabels(['{:,.2%}'.format(x) for x in plt.gca().get_yticks()])
        plt.legend([p2,p1],['Statewide Crashes ({})'.format(Crash_DF.shape[0]),'Site-Specific Crashes ({})'.format(len(Selection))],loc=2,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.tight_layout()
        plt.savefig(png_out,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close()
def SHSP_TimeTrend_Contour(DF,PNGName,Title):
    import warnings
    warnings.filterwarnings('ignore')
    import matplotlib.pyplot as plt
    DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in DF.Date]
    TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
    TimeOrder.reverse()
    DF['DayName'] = [d.weekday_name for d in DF.Date]
    DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    DF['Month']  = [d.strftime('%b') for d in DF.Date]
    MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    if 1<2:
        df0 = pd.DataFrame(DF.groupby(['Month','DayName','Time'])['KAB'].aggregate(sum))
        df0 = df0.unstack('Time').T
        df0.index = df0.index.droplevel(0)
        arr = []
        for m in MonthOrder:
            for d in DayOrder:
                arr.append((m,d))
        cs = pd.MultiIndex.from_tuples(arr)
        df = pd.DataFrame(columns=cs,index=TimeOrder)
        for i1 in df.index:
            for j1 in df.columns:
                try:
                    df.set_value(i1,j1,int(df0.loc[i1,j1]))
                except:
                    pass
        df = df.fillna(0)
        df = df.loc[TimeOrder]
        L = []
        for l in list(df.values):
            L.extend(l)
        v_max = max(L)
        v_min = min(L)
        df = df/v_max
        v_max = 1
        v_min = 0
    
        plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
        ax1 = plt.subplot(211)
    
        df1 = df[MonthOrder[0:6]]
        #pl1 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl1 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        xposition = [7*(i1)-0.5 for i1 in range(1,6)]
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[1][0:2] + '  ' + f[0],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.gca().xaxis.tick_top()
        plt.yticks(range(1,len(df1.index)),list(df1.index)[1:],rotation=0,fontsize=6)
        
        plt.grid()
        df1 = df[MonthOrder[6:13]]
        ax2 = plt.subplot(212)
        #pl2 = plt.contourf(df1, cmap=plt.cm.Reds,corner_mask=True ,alpha=1)
        pl2 = plt.imshow(df1, cmap=plt.cm.Reds,alpha=1,aspect='auto')
        for xc in xposition:
            plt.axvline(x=xc, color='k', linestyle='--')
        xl = [{True:f[0] + '  ' + f[1][0:2],False:f[1][0:2]}[f[1][0:2]=='Th'] for f in df1.columns]
        plt.xticks(range(len(list(df1))),xl,rotation=90,fontsize=6)
        plt.yticks(range(len(df1.index)),df1.index,rotation=0,fontsize=6)
        plt.grid()
        #plt.gcf().subplots_adjust(right=0.8)
        #cbar_ax = plt.gcf().add_axes([0.85, 0.15, 0.05, 0.7])
        #plt.colorbar(pl1, cax=cbar_ax)
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.suptitle(Title,y=0.99)
        #plt.tight_layout()
        plt.savefig(PNGName,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close('all')
        del ax1
        del ax2
        del pl1
        del pl2

print(strftime("%Y-%m-%d %H:%M:%S"))

2019-05-16 10:16:51
2019-05-16 10:16:51


In [48]:
path_to_domains = r'\\chcfpp01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\10.Extension\NM_Domains.xlsx'
df = pd.read_excel(path_to_domains)
def TranslateDomains_CSV(DF,DomainCSV,subset=[]):
    df = pd.DataFrame(columns=['field_name'])
    if os.path.basename(DomainCSV).split('.')[-1].lower()=='csv':
        df = pd.read_csv(DomainCSV)
    if os.path.basename(DomainCSV).split('.')[-1].lower() in ['xls','xlsx']:
        df = pd.read_excel(DomainCSV)
    if len(subset)==0:
        subset = DF.columns
    else:
        subset = [c for c in subset if c in DF.columns]
    for c in subset:
        if c in df.field_name.unique():
            idf = df[df.field_name=='Class'][['code','description']]
            idf.index = idf.code
            d = idf.description.to_dict()
            if len(d)>0:
                DF[c] = DF[c].apply(lambda x:'{}. {}'.format(x,d[x]) if x in d.keys() else x)
    return(DF)
Crash_DF['Class_T'] = TranslateDomains_CSV(Crash_DF[['Class']].fillna(99).astype(int),path_to_domains)
df = pd.DataFrame(Crash_DF.groupby(['Class_T','KABCO']).size()).unstack().fillna(0).astype(int)
df.columns = df.columns.droplevel(0)
df = df.sort_values(['K','A','B'],ascending=False)
df = df[['K','A','B']]
df

KABCO,K,A,B
Class_T,,,
4. Other Vehicle,476,2835,8099
1. Overturn,372,665,2138
3. Pedestrian,311,451,854
12. Rollover,160,226,528
10. Fixed Object,156,579,2080
99. Left Blank,82,106,257
8. Pedalcyclist,25,136,676
2. Other non-collision,23,129,494
5. Vehicle on the roadway,16,31,99


In [15]:
F = ['PedInv','PECinv','Druginv','Alcinv','MCinv','TRKinv','HZinv','OnCurve','TribalJurisdiction','Int_Related']
Des =  ['Pedestrian Involved', 'Pedalcyclist Involved','Drug Involved','Alcohol Involved','Motorcylce Involved','Truck Involved',
'Hazardous Material Involved','On the Curve','Tribal Jurisdiction','Intersection Related']
def EASeverity(All_Crashes_DF,SelectionIndex,EAFields = [],Descriptions = [],SeverityField='KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    for f in EAFields:
        All_Crashes_DF[f] = All_Crashes_DF[f].apply(lambda x:1 if x==1 else 0)

    All_DF = Crash_DF.groupby(SeverityField)[EAFields].sum().T
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()
    
    
    Sel_DF = Crash_DF.loc[SelectionIndex].groupby(SeverityField)[EAFields].sum().T
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    All_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    Sel_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,Crash_DF[Crash_DF[SeverityField]==label].shape[0]),
                            'Selected {} Crashes ({})'.format(label,Crash_DF.loc[SelectionIndex][Crash_DF[SeverityField]==label].shape[0])],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()

def CrashTypeSeverity(All_Crashes_DF,SelectionIndex,TypeField = 'Class_T',SeverityField = 'KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    All_DF = pd.DataFrame(All_Crashes_DF.groupby([TypeField,SeverityField]).size()).unstack().fillna(0).astype(int)
    All_DF.columns = All_DF.columns.droplevel(0)
    All_DF = All_DF[Order]
    All_DF = All_DF.sort_values(Order,ascending=False)
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()

    Sel_DF = pd.DataFrame(All_Crashes_DF.loc[SelectionIndex].groupby([TypeField,SeverityField]).size()).unstack()
    Sel_DF.columns = Sel_DF.columns.droplevel(0)
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,All_DF[label].sum()),
                            'Selected {} Crashes ({})'.format(label,Sel_DF[label].sum())],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()
    plt.clf()
print(strftime("%Y-%m-%d %H:%M:%S"))
f = IntProgress(min=0,max=Res_DF.shape[0],value=0)
display(f)
for i,r in Res_DF.iterrows():
    CrashTypeSeverity(Crash_DF,Res_DF.All_CIDs.loc[i],'Class_T','KABCO',['K','A','B'], PNGOut='Site_{}_TypeSeverity.png'.format(r.Site_ID),
                  title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
    EASeverity(Crash_DF,Res_DF.All_CIDs.loc[i],F,Des,SeverityField='KABCO',Order = ['K','A','B'], PNGOut='Site_{}_EASeverity.png'.format(r.Site_ID),
               title='{} - {} - {} - {}'.format(r.District,r.StateLocal,r.UrbanRural,r.Dis_Label))
    f.value+=1
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-04-25 11:54:08


IntProgress(value=0, max=447)

2019-04-25 13:24:04


In [24]:
F = ['PedInv','PECinv','Druginv','Alcinv','MCinv','TRKinv','HZinv','OnCurve','TribalJurisdiction','Int_Related']
Des =  ['Pedestrian Involved', 'Pedalcyclist Involved','Drug Involved','Alcohol Involved','Motorcylce Involved','Truck Involved',
'Hazardous Material Involved','On the Curve','Tribal Jurisdiction','Intersection Related']
def EASeverity(All_Crashes_DF,SelectionIndex,EAFields = [],Descriptions = [],SeverityField='KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    for f in EAFields:
        All_Crashes_DF[f] = All_Crashes_DF[f].apply(lambda x:1 if x==1 else 0)

    All_DF = Crash_DF.groupby(SeverityField)[EAFields].sum().T
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()
    
    
    Sel_DF = Crash_DF.loc[SelectionIndex].groupby(SeverityField)[EAFields].sum().T
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    All_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    Sel_DF.rename(index={v:k for v,k in zip(EAFields,Descriptions)},inplace=True)
    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,Crash_DF[Crash_DF[SeverityField]==label].shape[0]),
                            'Selected {} Crashes ({})'.format(label,Crash_DF.loc[SelectionIndex][Crash_DF[SeverityField]==label].shape[0])],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()

def CrashTypeSeverity(All_Crashes_DF,SelectionIndex,TypeField = 'Class_T',SeverityField = 'KABCO',Order = ['K','A','B'], PNGOut='out.png',title=''):
    plt.figure(figsize=(13, 7), dpi=300, facecolor='w', edgecolor='k')
    All_DF = pd.DataFrame(All_Crashes_DF.groupby([TypeField,SeverityField]).size()).unstack().fillna(0).astype(int)
    All_DF.columns = All_DF.columns.droplevel(0)
    All_DF = All_DF[Order]
    All_DF = All_DF.sort_values(Order,ascending=False)
    for c in All_DF.columns:
        All_DF['_'.join([c,'pct'])] = All_DF[c]/All_DF[c].sum()

    Sel_DF = pd.DataFrame(All_Crashes_DF.loc[SelectionIndex].groupby([TypeField,SeverityField]).size()).unstack()
    Sel_DF.columns = Sel_DF.columns.droplevel(0)
    Sel_DF = Sel_DF.loc[All_DF.index]
    Sel_DF = Sel_DF.T.loc[All_DF.columns].T
    Sel_DF = Sel_DF.fillna(0).astype(int)
    for c in Sel_DF.columns:
        Sel_DF['_'.join([c,'pct'])] = Sel_DF[c]/Sel_DF[c].sum()

    for i,sev in enumerate(['_'.join([c,'pct']) for c in Order]):
        label = Order[i]
        plt.subplot(len(Order),1,i+1)
        V = [v1-v2 for v1,v2 in zip(Sel_DF[sev],All_DF[sev])]
        my_cmap = matplotlib.cm.get_cmap('RdYlGn_r')
        my_norm = matplotlib.colors.Normalize(vmin=min(V), vmax=max(V))
        p1 = plt.bar(range(len(Sel_DF)),Sel_DF[sev],align='center',color=my_cmap(my_norm(V)))
        p2, = plt.plot(range(len(All_DF)),All_DF[sev],'-o',color='green')
        plt.gca().set_ylim([0,1])
        plt.xticks(range(len(Sel_DF)),[],rotation=90)
        plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in plt.gca().get_yticks()])
        #plt.title(sev,position=(0.5, 0.8))
        plt.legend([p2,p1],['All {} Crashes ({})'.format(label,All_DF[label].sum()),
                            'Selected {} Crashes ({})'.format(label,Sel_DF[label].sum())],loc=1,fancybox=True,framealpha=0.5,prop={'size': 9})
        plt.grid()
    plt.xticks(range(len(Sel_DF)),Sel_DF.index,rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0.1)
    plt.suptitle(title)
    plt.savefig(PNGOut,transparent=False,dpi=400, bbox_inches='tight', pad_inches=0)
    for i in Order:
        plt.close()
    plt.clf()
print(strftime("%Y-%m-%d %H:%M:%S"))

print(strftime("%Y-%m-%d %H:%M:%S"))

2019-05-16 09:57:09
2019-05-16 09:57:09
